Reference: https://github.com/dh1105/Sentence-Entailment

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers
# !pip install sentencepiece
## !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [3]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
import numpy as np

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
task_a_hyp = "This is hate speech."
task_b_hyp = "This is targeted hate."
task_c_hyp = "This is aggressive hate."
f = open('Data/HatEval/hateval2019_en_train.csv')
f_a = open('Data/HatEval/bert_nli_train.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split(',')
  tweet_id = row[0].strip()
  tweet_text = ','.join(row[1:-3]).strip()
  is_hate = row[-3].strip()
  is_targeted = row[-2].strip()
  is_ag = row[-1].strip()
  if is_hate == '0':
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    if is_ag == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
f.close()
f_a.close()

In [6]:
train_df = pd.read_csv('./Data/HatEval/bert_nli_train.csv', delimiter='\t')

In [7]:
train_df = train_df.sample(frac=1)

In [8]:
train_df

,tweet_id,gold_label,sentence1,sentence2
12208,6087,entailment,NOTHING BUT A MONEY GRUBBING RUSSIAN CUNT WHOR...,This is hate speech.
12202,6085,entailment,"I miss u bUT ALSO FUCK U, CUNT WHORE BITCH",This is hate speech.
7233,4224,contradiction,Germany reaches first deal with #Spain to retu...,This is hate speech.
16327,9072,entailment,@NiccoMontano @pokipokiabq Relinquish the belt...,This is hate speech.
7385,4314,contradiction,Democrat-run city building 6-foot fence around...,This is hate speech.
...,...,...,...,...
2167,1408,contradiction,Fifty years from now I’ll be the woman being w...,This is hate speech.
3848,2329,contradiction,"Sister Patricia Fox, the Australian nun whose ...",This is hate speech.
6752,3968,contradiction,Frontex can help...w-accounting.ðŸ˜¬Migrant sh...,This is targeted hate.
3272,2015,entailment,Looks like a bread line in Africa.. Can #Spain...,This is hate speech.


In [9]:
task_a_hyp = "This is hate speech."
task_b_hyp = "This is targeted hate."
task_c_hyp = "This is aggressive hate."
f = open('Data/HatEval/hateval2019_en_dev.csv')
f_a = open('Data/HatEval/bert_nli_val.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split(',')
  tweet_id = row[0].strip()
  tweet_text = ','.join(row[1:-3]).strip()
  is_hate = row[-3].strip()
  is_targeted = row[-2].strip()
  is_ag = row[-1].strip()
  if is_hate == '0':
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    if is_ag == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
f.close()
f_a.close()

In [10]:
val_df = pd.read_csv('./Data/HatEval/bert_nli_val.csv', delimiter='\t')

In [11]:
val_df = val_df.sample(frac=1)

In [12]:
val_df

,tweet_id,gold_label,sentence1,sentence2
272,18357,contradiction,We Congratulate Justice Tahira Safdar to be th...,This is hate speech.
249,18340,entailment,Here is Why Cortez is an Idiot 1) Border Cros...,This is hate speech.
770,18633,entailment,Meanwhile in Israel * African refugees are bei...,This is hate speech.
1549,19039,entailment,you ready baby ? ~ Im going to fuck you and yo...,This is targeted hate.
912,18717,contradiction,@atrupar Thanks MUCH for creating this hearing...,This is aggressive hate.
...,...,...,...,...
17,18212,entailment,@CBPArizona This is insane. 125 last weekend. ...,This is hate speech.
211,18322,entailment,About 25% of illegal crossers have a criminal ...,This is hate speech.
247,18338,entailment,In addition to possibly losing a few million h...,This is aggressive hate.
1288,18902,entailment,@Elaine61676401 @Leah49274379 @PattyArquette @...,This is targeted hate.


In [13]:
task_a_hyp = "This is hate speech."
task_b_hyp = "This is targeted hate."
task_c_hyp = "This is aggressive hate."
f = open('Data/HatEval/hateval2019_en_test.csv')
f_a = open('Data/HatEval/bert_nli_test.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split(',')
  tweet_id = row[0].strip()
  tweet_text = ','.join(row[1:-3]).strip()
  is_hate = row[-3].strip()
  is_targeted = row[-2].strip()
  is_ag = row[-1].strip()
  if is_hate == '0':
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    if is_ag == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
f.close()
f_a.close()

In [14]:
test_df = pd.read_csv('./Data/HatEval/bert_nli_test.csv', delimiter='\t')

In [15]:
test_df = test_df.sample(frac=1)

In [16]:
test_df

,tweet_id,gold_label,sentence1,sentence2
3793,31211,contradiction,"swear, next bitch i fuck ima get her pregnant ...",This is targeted hate.
3524,30831,contradiction,@JEEZPHOEBS Smoke weed bitch,This is hate speech.
3927,34470,entailment,If I trick a bitch and let her think I'm Obama...,This is hate speech.
936,34468,entailment,#Illegals are not immunized therefore bringing...,This is aggressive hate.
5093,31060,contradiction,@unfuckwitable27 no bitch idk about my family ...,This is aggressive hate.
...,...,...,...,...
1140,30688,contradiction,"@ICEgov GOD BLESS ICE, THANK YOU FOR GETTING I...",This is aggressive hate.
730,31904,contradiction,"@feingold32 @bodysouls 1) #BuildTheWall, 2) #S...",This is targeted hate.
3134,33112,contradiction,"if your comeback is just saying the words ""bit...",This is hate speech.
1707,32908,contradiction,@TwitchyTeam Hey @twitter: #IllegalAliens are ...,This is targeted hate.


In [17]:
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

In [18]:
train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

In [19]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [20]:
test_df['sentence1'] = test_df['sentence1'].astype(str)
test_df['sentence2'] = test_df['sentence2'].astype(str)

In [21]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
test_df = test_df[(test_df['sentence1'].str.split().str.len() > 0) & (test_df['sentence2'].str.split().str.len() > 0)]

In [22]:
train_df

,tweet_id,gold_label,sentence1,sentence2
12208,6087,entailment,NOTHING BUT A MONEY GRUBBING RUSSIAN CUNT WHOR...,This is hate speech.
12202,6085,entailment,"I miss u bUT ALSO FUCK U, CUNT WHORE BITCH",This is hate speech.
7233,4224,contradiction,Germany reaches first deal with #Spain to retu...,This is hate speech.
16327,9072,entailment,@NiccoMontano @pokipokiabq Relinquish the belt...,This is hate speech.
7385,4314,contradiction,Democrat-run city building 6-foot fence around...,This is hate speech.
...,...,...,...,...
2167,1408,contradiction,Fifty years from now I’ll be the woman being w...,This is hate speech.
3848,2329,contradiction,"Sister Patricia Fox, the Australian nun whose ...",This is hate speech.
6752,3968,contradiction,Frontex can help...w-accounting.ðŸ˜¬Migrant sh...,This is targeted hate.
3272,2015,entailment,Looks like a bread line in Africa.. Can #Spain...,This is hate speech.


In [23]:
val_df

,tweet_id,gold_label,sentence1,sentence2
272,18357,contradiction,We Congratulate Justice Tahira Safdar to be th...,This is hate speech.
249,18340,entailment,Here is Why Cortez is an Idiot 1) Border Cros...,This is hate speech.
770,18633,entailment,Meanwhile in Israel * African refugees are bei...,This is hate speech.
1549,19039,entailment,you ready baby ? ~ Im going to fuck you and yo...,This is targeted hate.
912,18717,contradiction,@atrupar Thanks MUCH for creating this hearing...,This is aggressive hate.
...,...,...,...,...
17,18212,entailment,@CBPArizona This is insane. 125 last weekend. ...,This is hate speech.
211,18322,entailment,About 25% of illegal crossers have a criminal ...,This is hate speech.
247,18338,entailment,In addition to possibly losing a few million h...,This is aggressive hate.
1288,18902,entailment,@Elaine61676401 @Leah49274379 @PattyArquette @...,This is targeted hate.


In [24]:
test_df

,tweet_id,gold_label,sentence1,sentence2
3793,31211,contradiction,"swear, next bitch i fuck ima get her pregnant ...",This is targeted hate.
3524,30831,contradiction,@JEEZPHOEBS Smoke weed bitch,This is hate speech.
3927,34470,entailment,If I trick a bitch and let her think I'm Obama...,This is hate speech.
936,34468,entailment,#Illegals are not immunized therefore bringing...,This is aggressive hate.
5093,31060,contradiction,@unfuckwitable27 no bitch idk about my family ...,This is aggressive hate.
...,...,...,...,...
1140,30688,contradiction,"@ICEgov GOD BLESS ICE, THANK YOU FOR GETTING I...",This is aggressive hate.
730,31904,contradiction,"@feingold32 @bodysouls 1) #BuildTheWall, 2) #S...",This is targeted hate.
3134,33112,contradiction,"if your comeback is just saying the words ""bit...",This is hate speech.
1707,32908,contradiction,@TwitchyTeam Hey @twitter: #IllegalAliens are ...,This is targeted hate.


In [25]:
test_df_a = test_df[test_df.sentence2 == "This is hate speech."].sort_values(by=['tweet_id'])
test_df_b = test_df[test_df.sentence2 == "This is targeted hate."].sort_values(by=['tweet_id'])
test_df_c = test_df[test_df.sentence2 == "This is aggressive hate."].sort_values(by=['tweet_id'])

In [26]:
print(len(test_df_a))
print(len(test_df_b))
print(len(test_df_c))

3000
1260
1260


In [27]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df, test_df_a, test_df_b, test_df_c):
    self.label_dict = {'contradiction': 0, 'entailment': 1}

    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.test_df_a = test_df_a
    self.test_df_b = test_df_b
    self.test_df_c = test_df_c
    self.base_path = 'mnli-data-hateval'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.test_data_a = None
    self.test_data_b = None
    self.test_data_c = None
    self.init_data()

  def init_data(self):
    #Saving takes too much RAM
    
    if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
      print("Found training data")
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
        self.train_data = pickle.load(f)
    else:
      self.train_data = self.load_data(self.train_df)
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
        pickle.dump(self.train_data, f)
    if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
      print("Found val data")
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
        self.val_data = pickle.load(f)
    else:
      self.val_data = self.load_data(self.val_df)
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
        pickle.dump(self.val_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data.pkl')):
      print("Found test data")
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'rb') as f:
        self.test_data = pickle.load(f)
    else:
      self.test_data = self.load_data(self.test_df)
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'wb') as f:
        pickle.dump(self.test_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_a.pkl')):
      print("Found test data a")
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'rb') as f:
        self.test_data_a = pickle.load(f)
    else:
      self.test_data_a = self.load_data(self.test_df_a)
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'wb') as f:
        pickle.dump(self.test_data_a, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_b.pkl')):
      print("Found test data b")
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'rb') as f:
        self.test_data_b = pickle.load(f)
    else:
      self.test_data_b = self.load_data(self.test_df_b)
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'wb') as f:
        pickle.dump(self.test_data_b, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_c.pkl')):
      print("Found test data c")
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'rb') as f:
        self.test_data_c = pickle.load(f)
    else:
      self.test_data_c = self.load_data(self.test_df_c)
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'wb') as f:
        pickle.dump(self.test_data_c, f)
    # self.train_data = self.load_data(self.train_df)
    # self.val_data = self.load_data(self.val_df)
    # self.test_data = self.load_data(self.test_df)
  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_a = DataLoader(
      self.test_data_a,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_b = DataLoader(
      self.test_data_b,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_c = DataLoader(
      self.test_data_c,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c

In [28]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df, test_df_a, test_df_b, test_df_c)

16566
1854
5520
3000
1260
1260


In [29]:
train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c = mnli_dataset.get_data_loaders(batch_size=32)

In [30]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [31]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [32]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [34]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [40]:
import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)
  #model.load_state_dict(torch.load('./WeightsHateval/nli_4.pt'))
  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      if batch_idx%50 == 0:
          print(batch_idx, len(train_loader))
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    torch.save(model.state_dict(),os.path.join('./WeightsHateval', 'nli_new_{}.pt'.format(epoch)))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [42]:
import time
def test(model, test_loader):
  model.load_state_dict(torch.load('WeightsHateval/nli_new_4.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  return [0 if preds[i][0] > preds[i][1] else 1 for i in range(len(preds))], acts

In [41]:
train(model, train_loader, val_loader, optimizer)

0 518
50 518
100 518
150 518
200 518
250 518
300 518
350 518
400 518
450 518
500 518
Epoch 1: train_loss: 0.3132 train_acc: 0.8655 | val_loss: 0.6849 val_acc: 0.7717
00:06:32.15
0 518
50 518
100 518
150 518
200 518
250 518
300 518
350 518
400 518
450 518
500 518
Epoch 2: train_loss: 0.1078 train_acc: 0.9637 | val_loss: 0.7527 val_acc: 0.7772
00:06:32.20
0 518
50 518
100 518
150 518
200 518
250 518
300 518
350 518
400 518
450 518
500 518
Epoch 3: train_loss: 0.0545 train_acc: 0.9836 | val_loss: 0.7831 val_acc: 0.7736
00:06:32.14
0 518
50 518
100 518
150 518
200 518
250 518
300 518
350 518
400 518
450 518
500 518
Epoch 4: train_loss: 0.0425 train_acc: 0.9881 | val_loss: 1.0025 val_acc: 0.7691
00:06:32.53
0 518
50 518
100 518
150 518
200 518
250 518
300 518
350 518
400 518
450 518
500 518
Epoch 5: train_loss: 0.0340 train_acc: 0.9905 | val_loss: 0.9656 val_acc: 0.7820
00:06:32.39


In [43]:
preds, acts = test(model, test_loader)

test_loss: 2.0342 test_acc: 0.6268
00:01:22.44


In [44]:
acc = 0
for i in range(len(preds)):
    acc += 1 if preds[i] == acts[i] else 0
print("Overall test accuracy is", acc/len(acts))

Overall test accuracy is 0.6264492753623189


In [45]:
preds_a, acts_a = test(model, test_loader_a)

test_loss: 2.8607 test_acc: 0.5104
00:00:44.79


In [57]:
ent_acts_a = 0
cont_acts_a = 0
correct_ents_a = 0
correct_conts_a = 0
for i in range(len(preds_a)):
    if acts_a[i] == 1:
        ent_acts_a += 1
        if preds_a[i] == 1:
            correct_ents_a += 1
    else:
        cont_acts_a += 1
        if preds_a[i] == 0:
            correct_conts_a += 1
            
print("Total hate examples:", ent_acts_a)
print("Correctly predicted hate examples:", correct_ents_a)
print("Accuracy for predicting hate examples:", correct_ents_a/ent_acts_a)
print("Total non hate examples:", cont_acts_a)
print("Correctly predicted non hate examples:", correct_conts_a)
print("Accuracy for predicting non hate examples:", correct_conts_a/cont_acts_a)

Total hate examples: 1260
Correctly predicted hate examples: 1209
Accuracy for predicting hate examples: 0.9595238095238096
Total non hate examples: 1740
Correctly predicted non hate examples: 323
Accuracy for predicting non hate examples: 0.18563218390804598


In [55]:
#CLASS A:
tp_a = 0
fp_a = 0
tn_a = 0
fn_a = 0
for i in range(len(preds_a)):
    if acts_a[i] == 1:
        if preds_a[i] == 1:
            tp_a += 1
        else:
            fn_a += 1
    else:
        if preds_a[i] == 0:
            tn_a += 1
        else:
            fp_a += 1
accuracy_a = (tp_a + tn_a)/(tp_a + fp_a + fn_a + tn_a)
recall_a = tp_a/(tp_a + fn_a)
precision_a = tp_a/(tp_a + fp_a)
f1_a = (2*precision_a*recall_a)/(precision_a + recall_a)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_a, fp_a, fn_a, tn_a))
print("Recall:", recall_a)
print("Precsion:", precision_a)
print("F1:", f1_a)
print("Accuracy", accuracy_a)

TP: 1209, FP: 1417
FN: 51, TN: 323
Recall: 0.9595238095238096
Precsion: 0.4603960396039604
F1: 0.6222336592897582
Accuracy 0.5106666666666667


In [52]:
preds_b, acts_b = test(model, test_loader_b)

test_loss: 0.4461 test_acc: 0.8703
00:00:08.70


In [56]:
#CLASS B:
tp_b = 0
fp_b = 0
tn_b = 0
fn_b = 0
for i in range(len(preds_b)):
    if acts_b[i] == 1:
        if preds_b[i] == 1:
            tp_b += 1
        else:
            fn_b += 1
    else:
        if preds_b[i] == 0:
            tn_b += 1
        else:
            fp_b += 1
accuracy_b = (tp_b + tn_b)/(tp_b + fp_b + fn_b + tn_b)
recall_b = tp_b/(tp_b + fn_b)
precision_b = tp_b/(tp_b + fp_b)
f1_b = (2*precision_b*recall_b)/(precision_b + recall_b)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_b, fp_b, fn_b, tn_b))
print("Recall:", recall_b)
print("Precsion:", precision_b)
print("F1:", f1_b)
print("Accuracy", accuracy_b)

TP: 470, FP: 107
FN: 59, TN: 624
Recall: 0.888468809073724
Precsion: 0.8145580589254766
F1: 0.8499095840867994
Accuracy 0.8682539682539683


In [54]:
ent_acts_b = 0
cont_acts_b = 0
correct_ents_b = 0
correct_conts_b = 0
for i in range(len(preds_b)):
    if acts_b[i] == 1:
        ent_acts_b += 1
        if preds_b[i] == 1:
            correct_ents_b += 1
    else:
        cont_acts_b += 1
        if preds_b[i] == 0:
            correct_conts_b += 1
            
print("Total targeted hate examples:", ent_acts_b)
print("Correctly predicted targeted hate examples:", correct_ents_b)
print("Accuracy for predicting targeted hate examples:", correct_ents_b/ent_acts_b)
print("Total non targeted offensive examples:", cont_acts_b)
print("Correctly predicted non targeted hate examples:", correct_conts_b)
print("Accuracy for predicting non targeted hate examples:", correct_conts_b/cont_acts_b)

Total targeted hate examples: 731
Correctly predicted targeted hate examples: 624
Accuracy for predicting targeted hate examples: 0.853625170998632
Total non targeted offensive examples: 529
Correctly predicted non targeted hate examples: 470
Accuracy for predicting non targeted hate examples: 0.888468809073724


In [49]:
preds_c, acts_c = test(model, test_loader_c)

test_loss: 1.6502 test_acc: 0.6604
00:00:08.67


In [58]:
#CLASS C:
tp_c = 0
fp_c = 0
tn_c = 0
fn_c = 0
for i in range(len(preds_c)):
    if acts_c[i] == 1:
        if preds_c[i] == 1:
            tp_c += 1
        else:
            fn_c += 1
    else:
        if preds_c[i] == 0:
            tn_c += 1
        else:
            fp_c += 1
accuracy_c = (tp_c + tn_c)/(tp_c + fp_c + fn_c + tn_c)
recall_c = tp_c/(tp_c + fn_c)
precision_c = tp_c/(tp_c + fp_c)
f1_c = (2*precision_c*recall_c)/(precision_c + recall_c)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_c, fp_c, fn_c, tn_c))
print("Recall:", recall_c)
print("Precsion:", precision_c)
print("F1:", f1_c)
print("Accuracy", accuracy_c)

TP: 352, FP: 186
FN: 242, TN: 480
Recall: 0.5925925925925926
Precsion: 0.654275092936803
F1: 0.6219081272084807
Accuracy 0.6603174603174603


In [59]:
ent_acts_c = 0
cont_acts_c = 0
correct_ents_c = 0
correct_conts_c = 0
for i in range(len(preds_c)):
    if acts_c[i] == 1:
        ent_acts_c += 1
        if preds_c[i] == 1:
            correct_ents_c += 1
    else:
        cont_acts_c += 1
        if preds_c[i] == 0:
            correct_conts_c += 1
            
print("Total aggressive hate examples:", ent_acts_c)
print("Correctly predicted aggressive hate examples:", correct_ents_c)
print("Accuracy for predicting aggressive hate examples:", correct_ents_c/ent_acts_c)
print("Total non aggressive examples:", cont_acts_c)
print("Correctly non aggressive examples:", correct_conts_c)
print("Accuracy for predicting non aggressive examples:", correct_conts_c/cont_acts_c)

Total aggressive hate examples: 594
Correctly predicted aggressive hate examples: 352
Accuracy for predicting aggressive hate examples: 0.5925925925925926
Total non aggressive examples: 666
Correctly non aggressive examples: 480
Accuracy for predicting non aggressive examples: 0.7207207207207207


In [62]:
actual_labels = []
predicted_labels = []
counter_a = 0
counter_b = 0
counter_c = 0
while counter_a < len(preds_a):
    if acts_a[counter_a] == 0:
        actual_labels.append((0, 0, 0))
        if preds_a[counter_a] == 0:
            predicted_labels.append((0, 0, 0))
        else:
            predicted_labels.append((1, 0, 0))
    else:
        actual_labels.append((1, acts_b[counter_b], acts_c[counter_c]))
        if preds_a[counter_a] == 0:
            predicted_labels.append((0, 0, 0))
        else:
            predicted_labels.append((1, preds_b[counter_b], preds_c[counter_c]))
        counter_b += 1
        counter_c += 1
    counter_a += 1
print("EMR i.e exact match ratio is", sum([predicted_labels[i] == actual_labels[i] for i in range(len(actual_labels))])/len(actual_labels))

EMR i.e exact match ratio is 0.341


In [64]:
print("Macro averaged F1 score is", (f1_a + f1_b + f1_c)/3)

Macro averaged F1 score is 0.698017123528346
